In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import np_utils 
from keras.datasets import cifar10
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

In [ ]:
seed = 5
np.random.seed(seed)

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_train /= 255
class_train = np_utils.to_categorical(y_train, 10)
class_test = np_utils.to_categorical(y_test, 10)


kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
resultados = []

a = np.zeros(5)
b = np.zeros(shape = (class_train.shape[0], 1))

for indice_treinamento, indice_teste in kfold.split(X_train, np.zeros(shape = (class_train.shape[0], 1))):
    #print('Índices treinamento: ', indice_treinamento, 'Índice teste', indice_teste)
    model = Sequential()
    #1ª camada de convolução
    model.add(Conv2D(32, (3,3), input_shape=(32, 32, 3), activation="relu")) #Conv2D(qdte de mapas de características, aumenta o valor de acoro com a dimensão da imagem)
    model.add(BatchNormalization()) #Normalização para as camadas de convuluções
    #MaxPooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #========================================================
    #2ª camada de convolução
    model.add(Conv2D(32, (3,3), activation="relu"))
    model.add(BatchNormalization()) #Normalização para as camadas de convuluções
    #MaxPooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #Flatten
    model.add(Flatten())
    #========================================================
    #1ª camada oculta
    model.add(Dense(units=128, activation="relu", )) # units=128/256/512
    model.add(Dropout(0.2))
    #========================================================
    #2ª camada oculta
    model.add(Dense(units=128, activation="relu", )) # units=128/256/512
    model.add(Dropout(0.2))
    model.add(Dense(units = 10, activation= "softmax"))


    model.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(X_train[indice_treinamento], class_train[indice_treinamento], batch_size = 128, epochs = 10)
    precisao = model.evaluate(X_train[indice_teste], class_train[indice_teste])
    resultados.append(precisao[1])
#===========================================================================================

#media = resultados.mean()
media = sum(resultados) / len(resultados)
media

170508288/170498071 [==============================] - 4s 0us/step
Epoch 1/10
265/313 [========================>.....] - ETA: 7s - loss: 1.6434 - accuracy: 0.4137

In [78]:
y_pred = model.predict(X_test)
y_pred

array([[0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.89843285,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

**Tabela comparativa entre o valor real e o preditivo...**

In [79]:
lista_resultados = []
for num in y_pred:
  y_pred = np.argmax(num)
  lista_resultados.append(y_pred)

In [80]:
import pandas as pd
tabela = pd.DataFrame()
tabela["Valores reais"] = y_test
tabela["Valores preditos"] = lista_resultados

In [81]:
tabela

,Valores reais,Valores preditos
0,7,7
1,2,2
2,1,8
3,0,0
4,4,4
...,...,...
9995,2,2
9996,3,3
9997,4,4
9998,5,8


In [82]:
tabela.to_csv("09-MNIST_resultado.csv")